In [1]:
import numpy as np
import pandas as pd
import math
import sys
import datetime as dt

In [2]:
# quarterly recipe data
data = pd.read_csv("../data/split data/2-SP500 Fundamental.csv") 
data_tmp = pd.DataFrame(data)
print(data_tmp.head())
print(data_tmp.tail())
print(data_tmp.shape[0])

   gvkey  datadate  fyearq  fqtr   tic                        conm datacqtr  \
0   1013  19900131    1990     1  ADCT  ADC TELECOMMUNICATIONS INC   1989Q4   
1   1013  19900430    1990     2  ADCT  ADC TELECOMMUNICATIONS INC   1990Q1   
2   1013  19900731    1990     3  ADCT  ADC TELECOMMUNICATIONS INC   1990Q2   
3   1013  19901031    1990     4  ADCT  ADC TELECOMMUNICATIONS INC   1990Q3   
4   1013  19910131    1991     1  ADCT  ADC TELECOMMUNICATIONS INC   1990Q4   

  datafqtr         rdq     actq   ...     oiadpq   revtq   saleq  teqq  \
0   1990Q1  19900223.0   85.600   ...      9.531  62.043  62.043   NaN   
1   1990Q2  19900529.0   87.438   ...      7.588  60.195  60.195   NaN   
2   1990Q3  19900829.0   89.083   ...      9.577  65.814  65.814   NaN   
3   1990Q4  19901213.0  102.525   ...     11.049  71.750  71.750   NaN   
4   1991Q1  19910221.0  107.228   ...     11.012  72.902  72.902   NaN   

    wcapq   xoprq  costat  mkvaltq   prccq  gsector  
0  58.483  49.240       I 

In [3]:
# stock name
stock_name = list(set(data_tmp['tic']))
for i in range(len(stock_name)-1):
    if(type(stock_name[i]) != str):
        del stock_name[i]  # delete nan in the list

"""
# transfer index into date
date = list(data_tmp['datadate'])
a = date
for i in range(len(date)):
    b = dt.strptime(str(date[i]), '%Y%m%d')
    a[i] = b.strftime('%Y-%m-%d')
series = data_tmp
series.index = a
print(series.head())
"""
print(data_tmp.columns.values)

['gvkey' 'datadate' 'fyearq' 'fqtr' 'tic' 'conm' 'datacqtr' 'datafqtr'
 'rdq' 'actq' 'apq' 'atq' 'ceqq' 'chq' 'cogsq' 'cshoq' 'dlcq' 'dlttq' 'dpq'
 'epspxq' 'invfgq' 'lctq' 'lltq' 'lseq' 'msaq' 'niq' 'oiadpq' 'revtq'
 'saleq' 'teqq' 'wcapq' 'xoprq' 'costat' 'mkvaltq' 'prccq' 'gsector']


In [4]:
# daily close price
data_fd = pd.read_csv("../data/split data/1-SP500 Price 1990 to 1999.csv", low_memory = False) # first decade
data_sd = pd.read_csv("../data/split data/1-SP500 Price 2000 to 2009.csv", low_memory = False) # second decade
data_td = pd.read_csv("../data/split data/1-SP500 Price 2010 to 2017.csv", low_memory = False) # third decade

data_fd = pd.DataFrame(data_fd[list(['datadate', 'tic', 'prccd', 'gsector'])])
data_sd = pd.DataFrame(data_sd[list(['datadate', 'tic', 'prccd', 'gsector'])])
data_td = pd.DataFrame(data_td[list(['datadate', 'tic', 'prccd', 'gsector'])])

print(data_fd.shape[0] + data_sd.shape[0] + data_td.shape[0])

data_daily = [data_fd, data_sd, data_td]
data_daily = pd.concat(data_daily)

print(data_daily.shape[0])


stock_name_daily = list(set(data_daily['tic']))
for i in range(len(stock_name_daily)-1):
    if(type(stock_name_daily[i]) != str):
        del stock_name_daily[i]  # delete nan in the list

6426271
6426271


In [5]:
len(stock_name_daily)

1825

In [6]:
# use trade date instead of report quarterly date
times = list(data_tmp['datadate']) # quarterly report date
for i in range(len(times)):
    quarter = (times[i] - int(times[i]/10000)*10000)
    if 1201 < quarter:
        times[i] = int(times[i]/10000 + 1)*10000 + 601
    if quarter <= 301:
        times[i] = int(times[i]/10000)*10000 + 601
    if 301 < quarter <= 601:
        times[i] = int(times[i]/10000)*10000 + 901
    if 601 < quarter <= 901:
        times[i] = int(times[i]/10000)*10000 + 1201
    if 901 < quarter <= 1201:
        times[i] = int(times[i]/10000 + 1)*10000 + 301
    time_tmp = times[i]
    year = int(time_tmp/10000)
    month = int((time_tmp - year*10000)/100)
    day = int(time_tmp - year*10000 - month*100)
    if(time_tmp < 20171114):
        while(dt.date(year, month, day).weekday() > 4):
            time_tmp = time_tmp + 1
            year = int(time_tmp/10000)
            month = int((time_tmp - year*10000)/100)
            day = int(time_tmp - year*10000 - month*100)
        times[i] = time_tmp

data_factor = pd.DataFrame({'datadate': times, 'tic': data_tmp['tic'], 'gsector':data_tmp['gsector'], 
                            'epspxq': data_tmp['epspxq']})
print(data_factor.head(16))
print(data_factor.tail(16))

    datadate  epspxq  gsector   tic
0   19900601    0.44     45.0  ADCT
1   19900903    0.36     45.0  ADCT
2   19901203    0.44     45.0  ADCT
3   19910301    0.49     45.0  ADCT
4   19910603    0.50     45.0  ADCT
5   19910902    0.49     45.0  ADCT
6   19911202    0.32     45.0  ADCT
7   19920302    0.34     45.0  ADCT
8   19920601   -0.09     45.0  ADCT
9   19920901    0.46     45.0  ADCT
10  19921201    0.56     45.0  ADCT
11  19930301    0.62     45.0  ADCT
12  19930601    0.39     45.0  ADCT
13  19930901    0.51     45.0  ADCT
14  19931201    0.33     45.0  ADCT
15  19940301    0.37     45.0  ADCT
       datadate  epspxq  gsector   tic
91200  20140602    0.12     20.0  ALLE
91201  20140901    0.38     20.0  ALLE
91202  20141201    0.53     20.0  ALLE
91203  20150302    0.66     20.0  ALLE
91204  20150601    0.37     20.0  ALLE
91205  20150901    0.48     20.0  ALLE
91206  20151201    0.67     20.0  ALLE
91207  20160301   -0.28     20.0  ALLE
91208  20160601    0.75     20.0  ALL

In [7]:
data_factor['prccd'] = [0]*data_factor.shape[0]
for i in stock_name:
    if (i in stock_name_daily):
        ind_quarter = np.where(data_factor['tic'] == i)[0]
        date_price_daily = data_daily.loc[data_daily['tic'] == i, list(['datadate', 'prccd'])]
        date_price_daily.index = range(date_price_daily.shape[0])
        date_max = max(date_price_daily['datadate'])
        date_min = min(date_price_daily['datadate'])
        for j in ind_quarter:
            if (data_factor.loc[j, 'datadate'] < date_min or data_factor.loc[j, 'datadate'] > date_max):
                data_factor.loc[j, 'prccd'] = np.nan
            else:
                if(data_factor.loc[j, 'datadate'] in set(date_price_daily['datadate'])):
                    ind_tmp = np.where(date_price_daily['datadate'] == data_factor.loc[j, 'datadate'])[0]
                    data_factor.loc[j, 'prccd'] = date_price_daily.loc[ind_tmp, 'prccd'].values
                else:
                    data_factor.loc[j, 'prccd'] = np.nan
 

In [8]:
print(data_factor)
writer = pd.ExcelWriter('trade date and price.xlsx')
data_factor.to_excel(writer,'Sheet1')
writer.save()

       datadate  epspxq  gsector   tic   prccd
0      19900601    0.44     45.0  ADCT  22.000
1      19900903    0.36     45.0  ADCT     NaN
2      19901203    0.44     45.0  ADCT  23.625
3      19910301    0.49     45.0  ADCT  32.000
4      19910603    0.50     45.0  ADCT  40.750
5      19910902    0.49     45.0  ADCT     NaN
6      19911202    0.32     45.0  ADCT  25.125
7      19920302    0.34     45.0  ADCT  25.750
8      19920601   -0.09     45.0  ADCT  35.000
9      19920901    0.46     45.0  ADCT  36.000
10     19921201    0.56     45.0  ADCT  45.500
11     19930301    0.62     45.0  ADCT  42.000
12     19930601    0.39     45.0  ADCT  54.750
13     19930901    0.51     45.0  ADCT  36.000
14     19931201    0.33     45.0  ADCT  35.375
15     19940301    0.37     45.0  ADCT  36.875
16     19940601    0.24     45.0  ADCT  41.375
17     19940901    0.36     45.0  ADCT  46.000
18     19941201    0.40     45.0  ADCT  41.000
19     19950301    0.46     45.0  ADCT  29.500
20     199506

In [9]:
# factors calculation function
def revenue_growth(df):
    # REVTQ (T) / REVTQ (T-1) -1
    rev = df['revtq']
    rev = round(rev.shift(1)/rev - 1, 3)
    return rev

def price_to_cash_flow_ratio(df2, df):
    # Adjusted Close Price * CSHOQ 
    # / [(NIQ (Net Income (Loss)) + DPQ (Depreciation and Amortization, Total) – (WCAPQ (T)- WCAPQ (T-1))]
    tmp = df['wcapq']
    tmp = round(tmp.shift(1) - tmp, 3)
    ptcfr = round(df2['prccd']*df['cshoq']/(df['niq'] + df['dpq'] - tmp), 3)
    return ptcfr
    
    
def cash_ratio(df):
    # [CHQ (Cash) + MSAQ (Marketable Security Adjustments)] / LCTQ (Current Liabilities, Total)
    chq_ratio = round((df['chq'] + df['msaq'])/df['lctq'], 3)
    return chq_ratio

def ev(df2 ,df):
    # EV = PRCCQ (Price Close, Quarter) * CSHOQ (Common Shares Outstanding) + DLTTQ (Long-Term Debt, Total) 
    # + DLCQ (Debt in Current Liabilities) - CHQ (Cash)
    ev = round(df2['prccd'] * df['cshoq'] + df['mkvaltq'] + df['dlttq'] + df['dlcq'] - df['chq'], 3)
    return ev

def ebitda(df):
    
    # EBITDA = REVTQ (Revenue, Total) – XOPRQ (Operating Expense, Total) 
    # + DPQ (Depreciation and Amortization, Total)
    ebitda = round(df['revtq'] - df['xoprq'] + df['dpq'], 3)
    return ebitda

def ev_to_cfo(df2, df):
    # [PRCCQ (Price Close, Quarter) * CSHOQ + DLTTQ (Long-Term Debt, Total) + DLCQ (Debt in Current Liabilities) - CHQ (Cash)]
    # /[(NIQ (Net Income (Loss)) + DPQ (Depreciation and Amortization, Total) – (WCAPQ (T)- WCAPQ (T-1))]
    tmp = df['wcapq']
    tmp = tmp.shift(1) - tmp
    
    #num = df['mkvaltq'] + df['dlttq'] + df['dlcq'] - df['chq']
    num2 = df['niq'] + df['dpq'] - tmp
    evtocfo = round(ev(df2, df)/num2, 3)
    return evtocfo

def roa(df):
    # NIQ (Net Income (Loss)) / ATQ (Assets, Total)
    return round(df['niq']/df['atq'], 3)

def roe(df):
    # NIQ (Net Income (Loss)) / CEQQ (Common/Ordinary Equity, Total)
    return round(df['niq']/df['ceqq'], 3)

def pe(df2, df):
    # Current market close price / EPSPXQ (Earnings Per Share (Basic) - Excluding Extraordinary Items)
    return round(df2['prccd']/df['epspxq'], 3)

def ps(df2, df):
    # Current market close price / [REVTQ (Revenue, Total) / CSHOQ (Common Shares Outstanding)]
    return round(df2['prccd'] / df['revtq'] / df['cshoq'], 3)

def net_profit_margin(df):
    # NIQ (Net Income (Loss)) / REVTQ (Revenue, Total)
    return round(df['niq']/df['revtq'], 3)

def gross_profit_margin(df):
    # [REVTQ (Revenue, Total) – COGSQ (Cost of Goods Sold)] / REVTQ (Revenue, Total)
    return round((df['revtq'] - df['cogsq'])/df['revtq'], 3)

def operating_margin(df):
    # OIADPQ (Operating Income After Depreciation, Quarterly) / SALEQ (Sales/Turnover (Net))
    return round(df['oiadpq']/df['saleq'], 3)

def price_to_book(df2, df):
    # Current market close price / {[ATQ (Assets, Total) – LTQ(Liabilities, Total)] / CSHOQ}
    #return round(df2['prccd'] / ((df['atq'] - df['ltq'])/df['cshoq']), 3)
    return round(df2['prccd'] / ((df['atq'] - df['lltq'])/df['cshoq']), 3)

def long_term_debt_to_total_asset(df):
    # DLTTQ (Long-Term Debt, Total) / ATQ (Assets, Total)
    return round(df['dlttq'] / df['atq'], 3)

def working_capital_ratio(df):
    # ACTQ (Current Assets, Total) / LCTQ (Current Liabilities, Total)
    return round(df['actq'] / df['lctq'], 3)

def debt_equity(df):
    # LTQ (Liabilities, Total) / CEQQ (Common/Ordinary Equity, Total)
    #return round(df['ltq']/df['ceqq'], 3)
    return round(df['lltq']/df['ceqq'], 3)


def quick_ratio(df):
    # [ ACTQ (Current Assets, Total) - INVTQ (Inventory, Finished Goods) ] / LCTQ (Current Liabilities, Total)
    #return round((df['actq'] - df['invtq']) / df['lctq'], 3)
    return round((df['actq'] - df['invfgq']) / df['lctq'], 3)

def days_sales_of_inventory(df):
    # INVTQ (Inventory, Finished Goods) / COGSQ (Cost of Goods Sold) * 365
    #return round(df['invtq'] / df['cogsq'] * 365, 3)
    return round(df['invfgq'] / df['cogsq'] * 365, 3)

def days_payable_outstanding(df):
    # APQ (Account Payable/Creditors, Trade) / COGSQ (Cost of Goods Sold) * 365
    return round(df['apq'] / df['cogsq'] * 365, 3)

In [10]:
# factors calculation
pd.set_option('mode.chained_assignment', None)
data_factor_store = pd.DataFrame(columns = ['datadate', 'gsector', 'tic', 'prccd',
                                            'revenue_growth', 'price_to_cash_flow_ratio', 
                                            'cash_ratio', 'ebitda', 'ev_to_cfo', 'epspxq', 
                                            'roa', 'roe', 'pe', 'ps', 'net_profit_margin', 
                                            'gross_profit_margin', 'operating_margin', 
                                            'price_to_book', 'long_term_debt_to_total_asset', 
                                            'working_capital_ratio', 'debt_equity', 'quick_ratio', 
                                            'days_sales_of_inventory', 'days_payable_outstanding'])
for i in stock_name:
    df = data_tmp.loc[data_tmp['tic'] == i, ]
    df2 = data_factor.loc[data_factor['tic'] == i, ]
    df2.loc[:, 'revenue_growth'] = list(revenue_growth(df))
    df2.loc[:, 'price_to_cash_flow_ratio'] = list(price_to_cash_flow_ratio(df2, df))
    df2.loc[:, 'cash_ratio'] = list(cash_ratio(df))
    df2.loc[:, 'ebitda'] = list(ebitda(df))
    df2.loc[:, 'ev_to_cfo'] = list(ev_to_cfo(df2, df))
    df2.loc[:, 'roa'] = list(roa(df))
    df2.loc[:, 'roe'] = list(roe(df))
    df2.loc[:, 'pe'] = list(pe(df2, df))
    df2.loc[:, 'ps'] = list(ps(df2, df))
    df2.loc[:, 'net_profit_margin'] = list(net_profit_margin(df))
    df2.loc[:, 'gross_profit_margin'] = list(gross_profit_margin(df))
    df2.loc[:, 'operating_margin'] = list(operating_margin(df))
    df2.loc[:, 'price_to_book'] = list(price_to_book(df2, df))
    df2.loc[:, 'long_term_debt_to_total_asset'] = list(long_term_debt_to_total_asset(df))
    df2.loc[:, 'working_capital_ratio'] = list(working_capital_ratio(df))
    df2.loc[:, 'debt_equity'] = list(debt_equity(df))
    df2.loc[:, 'quick_ratio'] = list(quick_ratio(df))
    df2.loc[:, 'days_sales_of_inventory'] = list(days_sales_of_inventory(df))
    df2.loc[:, 'days_payable_outstanding'] = list(days_payable_outstanding(df))
    data_factor_store = [data_factor_store, df2]
    data_factor_store = pd.concat(data_factor_store)


In [11]:
data_factor = data_factor_store
print(data_factor)
writer = pd.ExcelWriter('factor matrix with tradable date.xlsx')
data_factor.to_excel(writer,'Sheet1')
writer.save()

       cash_ratio  datadate  days_payable_outstanding  \
42671         NaN  19900903                   178.929   
42672         NaN  19901203                   189.723   
42673         NaN  19910301                   202.109   
42674         NaN  19910603                   207.436   
42675         NaN  19910902                   190.310   
42676         NaN  19911202                   224.611   
42677         NaN  19920302                   224.310   
42678         NaN  19920601                   330.029   
42679         NaN  19920901                   200.750   
42680         NaN  19921201                   258.276   
42681         NaN  19930301                   196.457   
42682         NaN  19930601                   253.270   
42683         NaN  19930901                   117.541   
42684         NaN  19931201                   121.352   
42685         NaN  19940301                   266.352   
42686         NaN  19940601                   295.292   
42687         NaN  19940901    